In [1]:
#Run this cell first by hitting the play button to load packages that you'll need in this pset
!pip install modin
from google.colab import drive
import matplotlib.pyplot as mplt
import numpy as np
import modin.pandas as pd
import seaborn as sb

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
Mounted at /content/drive


#Measure affective polarization

Our measure of affective polarization comes from the following two survey questions:

**`W4_PATA450a`: AFFECTIVE POLARIZATION DEMOCRATS FEELING THERMOMETER**

*We would like to get your feelings toward Democrats using something we call the feeling thermometer.
Ratings between 50 degrees and 100 degrees mean that you feel favorable and warm toward the group.
Ratings between 0 degrees and 50 degrees mean that you don’t feel favorable toward the group and that
you don’t care too much for that group. You would rate the group at the 50 degree mark if you don’t feel
particularly warm or cold toward the group.*
1. 0
2. ...
3. 100

**`W4_PATA450b`: AFFECTIVE POLARIZATION REPUBLICANS FEELING THERMOMETER**

*We now would like to get your feelings toward Republicans using the same feeling thermometer.*
1. 0
2. ...
3. 100

1. First, you will set up the data for analysis. Today, we will consider data from the survey to compute measures of affective polarization. Load in the survey data file `survey_data.csv` using [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) from the `pandas` module. Print the first rows in your dataset and and print the number of rows and columns in your merged dataset.

In [10]:
#read in the survey data

survey_df = pd.read_csv(f"/content/drive/My Drive/Stanford Summer Session/DATASCI 154/In-class/survey_data.csv")
print(survey_df.shape)
survey_df.head()

(1339, 179)


,id,W1_pid3,W1_gender,W1_educ,party7,partylean,age,agesq,female,raceeth,...,predicted_house_winner_gop,predicted_vote_share_gop,predicted_district_winner_gop,predicted_house_winner_dem,predicted_vote_share_dem,predicted_district_winner_dem,prediction_certainty,W4_PATA462_a,W4_PATA462_b,W4_PATA462_c
0,0b9fb6fe532e69827f935cad3c01da4f,1,1,5,1.0,Democrat,63,3969,0,White,...,True,55.0,False,False,45.0,True,8.0,1.0,1.0,5.0
1,4197507f12350e6658bc08c285d4613e,1,2,6,1.0,Democrat,44,1936,1,White,...,False,NaN,False,True,NaN,True,7.0,1.0,1.0,5.0
2,f8b531355a063ef2ea63f7ddaa67c306,3,1,6,3.0,Democrat,42,1764,0,Asian / other,...,False,43.0,False,True,57.0,True,7.0,1.0,2.0,5.0
3,777b5e86cae0379e2ee4b7f21df46586,3,1,4,5.0,Republican,63,3969,0,White,...,True,53.0,True,False,47.0,False,10.0,5.0,5.0,1.0
4,341a9697fb54fd94ebd14b1e332c8d77,1,1,5,2.0,Democrat,75,5625,0,White,...,False,40.0,False,True,60.0,True,8.0,1.0,1.0,5.0


2. We have a measure of participants' ideological leanings in a variable called `party7`. Values 1-3 indicate the participant has liberal views, while values 5-7 indicate the participant has conservative views. Using `survey_data.csv`, compute the measure of affective polarization as follows and briefly explain what this measure you are constructing captures:

- For participants with a value of `party7` between 1 and 3 (inclusive), compute the measure as `W4_PATA450a` - `W4_PATA450b`.
- For participants with a value of `party7` between 5 and 7 (inclusive), compute the measure as `W4_PATA450b` - `W4_PATA450a`.
- For all other participants, set the measure to `np.nan`.

In [16]:
def polarization_measure(row):
    party7 = row["party7"]
    a = row["W4_PATA450a"]
    b = row["W4_PATA450b"]

    if 1 <= party7 <= 3:
        return a - b
    elif party7 == 4:
        return np.nan
    else:
        return b - a

survey_df["measure"] = survey_df.apply(polarization_measure, axis=1)

In [17]:
survey_df

,id,W1_pid3,W1_gender,W1_educ,party7,partylean,age,agesq,female,raceeth,...,predicted_vote_share_gop,predicted_district_winner_gop,predicted_house_winner_dem,predicted_vote_share_dem,predicted_district_winner_dem,prediction_certainty,W4_PATA462_a,W4_PATA462_b,W4_PATA462_c,measure
0,0b9fb6fe532e69827f935cad3c01da4f,1,1,5,1.0,Democrat,63,3969,0,White,...,55.0,False,False,45.0,True,8.0,1.0,1.0,5.0,95.0
1,4197507f12350e6658bc08c285d4613e,1,2,6,1.0,Democrat,44,1936,1,White,...,NaN,False,True,NaN,True,7.0,1.0,1.0,5.0,94.0
2,f8b531355a063ef2ea63f7ddaa67c306,3,1,6,3.0,Democrat,42,1764,0,Asian / other,...,43.0,False,True,57.0,True,7.0,1.0,2.0,5.0,55.0
3,777b5e86cae0379e2ee4b7f21df46586,3,1,4,5.0,Republican,63,3969,0,White,...,53.0,True,False,47.0,False,10.0,5.0,5.0,1.0,56.0
4,341a9697fb54fd94ebd14b1e332c8d77,1,1,5,2.0,Democrat,75,5625,0,White,...,40.0,False,True,60.0,True,8.0,1.0,1.0,5.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,44f23a14a633922efcb691a5b935a26c,1,1,3,1.0,NaN,55,3025,0,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1335,fc188e912b1ae54656805ba72c4d742f,1,1,3,2.0,NaN,54,2916,0,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1336,0f4ca7fa88678d156bb493a894ea3502,3,2,3,3.0,NaN,43,1849,1,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337,f378fd5ea003691f51a923ac79d03ebc,1,2,4,1.0,NaN,46,2116,1,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**YOUR EXPLANATION HERE**

3. Write a function to compute two means: one for affective polarization among Republicans, and one for affective polarization among Democrats. Report the values you calculate. Hint: It may be helpful to use [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) and [`mean`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) with the parameter 'skipna' set to its default value of True.

In [ ]:
#function to compute means

YOUR CODE HERE